In [ ]:
%%javascript
$('#run_all_cells_below').click()


# Property Based Testing
## (Using Hypothesis)<br><br><br>
### Amsterdam Python Meetup
### 27 April 2017<br><br><br>
### Daniel Bradburn


* Property based testing

* Choosing properties

* Generating data

* Model based testing

* Django

* Real world examples

say we have a run length encoding function. We encode a string as characters and the number of consecutive occurrences of that character. let's just test this out with something simple

In [ ]:
def encode(input_string):
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [ ]:
encode('hello')

and we also have a decode function which reconstructs the string let's just check this function, let's use the output from the encode

In [ ]:
def decode(lst):
    return ''.join(c * n for c, n in lst)

In [ ]:
decode([('h', 1), ('e', 1), ('l', 2), ('o', 1)])

but it's probably best to formalize this in a unit test. I'm using pytest here, but you could use unittest or your favourite test runner, the principal is the same.

In [ ]:
def test_run_length_encode():
    input_data = "hello"
    expected = [('h', 1), ('e', 1), ('l', 2), ('o', 1)]
    actual = encode(input_data)
    assert actual == expected

In [ ]:
!py.test -k test_run_length_encode -q

In [ ]:
def test_run_length_decode():
    input_data = [('h', 1), ('e', 1), ('l', 2), ('o', 1)]
    expected = "hello"
    actual = decode(input_data)
    assert actual == expected

In [ ]:
!py.test -k test_run_length_decode -q

In [ ]:
import pytest

examples = ['hello', 'python', 'uhm...']

@pytest.mark.parametrize('input_data', examples)
def test_parameterized_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
!py.test -k test_parameterized_run_length_encode_decode -q

In [ ]:
import random, string

random.seed(0)

random_letter = lambda: random.choice(string.ascii_letters)
random_range = lambda m: range(random.randint(0, m))
random_word = lambda m: (random_letter() for i in random_range(m))
random_words = lambda n, m: (''.join(random_word(m)) for n in range(n))

@pytest.mark.parametrize('input_data', random_words(5, 10))
def test_fuzzed_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
!py.test -k test_fuzzed_run_length_encode_decode -q

In [ ]:
from hypothesis import strategies as st
from hypothesis import given

@given(st.text())
def test_property_based_run_length_encode_decode(input_data):
    assert decode(encode(input_data)) == input_data

In [ ]:
!py.test -k test_property_based_run_length_encode_decode -q

In [ ]:
def encode_fixed(input_string):
    count = 1
    prev = ''
    lst = []
    character = ''
    for character in input_string:
        if character != prev:
            if prev:
                lst.append((prev, count))
            count = 1
            prev = character
        else:
            count += 1
    else:
        lst.append((character, count))
    return lst

In [ ]:
@given(st.text())
def test_property_based_run_length_encode_fixed_decode(input_data):
    assert decode(encode_fixed(input_data)) == input_data

In [ ]:
!py.test -k test_fuzzed_run_length_encode_decode -q

In [ ]:
%%javascript
$('#clear_all_output').click()
